# Create Vs30 uniform grid csv file

##  Configuration

In [ ]:
# Import dependencies
import os
import glob
import pandas as pd

import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

import utils
from plots import gdf_epicentre, stations_plot
from openquake.commands.prepare_site_model import main as oq_site_model

## User input

In [ ]:
# Events list from ECD
event = '20150425_M7.8_Gorkha'

# Reference rupture
rupture = 'earthquake_rupture_model_USGS.xml'

# Grid parameters in km
# Indicate tuples with: (x_distance, y_distance, grid_spacing)
corners = [(50, 50, 10), (250, 250, 20), (500, 500, 50)]

# Define region CRS !!! (CRS UNITS MUST BE IN METERS)
# Check for reference (see https://spatialreference.org/)
region_crs = "EPSG:3857"  # EPSG:3857 - Web Mercator Auxiliary Sphere

# Clip sites based on country boundaries (use ISO3)
clip_countries = ['NPL'] # Empty list for the full map


## Create region bounding box

### Generate grid spacing with respect to a given epicentral coordinates

In [ ]:
# Select rupture file and get folders
rup_path = glob.glob(os.path.join('..', '*', '*', event, '*', rupture))[0]
folder = rup_path[:rup_path.find('Rupture')]

# Epicenter as GeoDataFrame
epicenter = gdf_epicentre(rup_path)

# Generate sites with a uniform grid spacing for each polygon
epicenter_meters = epicenter.to_crs(region_crs)
pointdf = utils.grid_spacing(corners, region_crs, epicenter_meters)

# Plot grid
ax = pointdf.plot(figsize=(12, 8))
epicenter.plot(ax=ax, marker='*', color='gold', markersize=300)

print('Grid size:', len(pointdf))

### Remove duplicate site when rounding to 5 decimal pleces (for OpenQuake) compatibility

### Plot grid and stations to verify contours

In [ ]:
st_path = glob.glob(os.path.join(folder, '*', 'stationlist_seismic.csv'))[0]
# st_path = glob.glob(os.path.join(folder, '*', 'stationlist_all.csv'))[0]

print('Ploting stations from:', os.path.basename(st_path))

# Get station data and plot it
stations = pd.read_csv(st_path)
title = f'''Recording stations {event.replace('DRAFT_', '')}'''
fig, ax = stations_plot(stations, title=title)

# Get boundaries to clip piont grid (pointdf)
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
if clip_countries:
    countries = world[world['iso_a3'].isin(clip_countries)]
    grid = pointdf.clip(countries)
else:
    grid = pointdf.copy()

assert len(grid) != 0, f'All sites filtered out. Review the clipping boundaries {clip_countries}'


# Convert site to DataFrame
sites_vs30 = pd.DataFrame({'lon':grid.geometry.x, 'lat':grid.geometry.y}).reset_index()
assert any(sites_vs30[['lon', 'lat']].duplicated()) == False, 'Duplicated sites. Check grid spacing'

# Plot sites
grid.plot(ax=ax,
          alpha=0.2,
          marker='s', 
          markersize=100,
          color='blue',
          zorder=0)

# To adjust the extents of the plot:
xmin, ymin, xmax, ymax = grid.total_bounds
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Add epicenter in plot
epicenter.plot(ax=ax, marker='*', color='gold', markersize=300)

print('Number of sites:', len(grid))

## Add VS30 data from USGS values

In [ ]:
# Vs30 reference file (no headers, use USGS format)
# When running from Wilson, it's possible to use the worldv30 dataset
# located at '/home/risk/sites_vs30/original/vs30mosaic.hdf5'
vs30_path = '../vs30mosaic.hdf5'

# Use OpenQuake site_model function
site_model = oq_site_model([vs30_path],
                            z1pt0=True,
                            z2pt5=True,
                            vs30measured=True,
                            sites_csv=sites_vs30,
                            output=None)
site_model = pd.DataFrame(site_model.array)
site_model.sids = 't_' + site_model.sids.astype(str)
site_model = (site_model.rename(columns={'sids':'custom_site_id'}).round(5)) # Round to 5 decimal places (OQ compatibility)

site_model

## Plot Vs30 grid

In [ ]:
# Get common matplotlib style
utils.matplotlib_style()

# Create GeoDataFrame with WGS84
gdf = gpd.GeoDataFrame(site_model,
                       geometry=gpd.points_from_xy(site_model.lon, site_model.lat),
                       crs='EPSG:4326')

# Plot station Vs30 value
fig, ax = plt.subplots(figsize=(12, 8)) # , alpha=0.5) # alpha not supported in OQ libraries
ax.set_aspect('equal')

gdf.plot(ax=ax,
         column='vs30',
         alpha=0.7,
         marker='s', 
         markersize=15,
         cmap='Greens',
         edgecolor='whitesmoke', 
         linewidth=0.5,
         label='USGS Vs30',
         legend=True,
         legend_kwds={'label': 'Vs30 (m/s)', 'pad':0.05, 'shrink':0.8,
                      'orientation': 'horizontal'})


# Add default basemap: Stamen Terrain (uses Web Mercator --> epsg=3857)
# Additional basemaps available at:
# https://contextily.readthedocs.io/en/latest/providers_deepdive.html#What-is-this-%E2%80%9Cprovider%E2%80%9D-object-?
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron,
                crs=gdf.crs, alpha=0.5)

ax.legend(loc='lower left')
ax.set_title(f'Vs30 with variable grid {corners}', 
             fontsize=14)

# Add epicenter in plot
epicenter.plot(ax=ax, marker='*', color='gold', markersize=300)

## Save files

In [ ]:
# Folder to wirte outputs
out_folder = os.path.join(folder, 'OpenQuake_gmfs')

# Save file
cols = ['custom_site_id', 'lon', 'lat', 'vs30','vs30measured','z1pt0','z2pt5']
save_as = f'site_model.csv'
out_path = os.path.join(out_folder, save_as)
site_model[cols].to_csv(out_path, encoding='utf-8', index=False)
print('\n Saving site_model in:\n', out_path)

# Save figure
out_path = os.path.join(out_folder, 'site_model.png')
fig.savefig(out_path, facecolor="w", 
            dpi=100,
            bbox_inches='tight')
print(out_path)